# Wines Points prediction 

Submission Date : 3.6.2023
Task: Predict the wine score given the inputs
Instructions:
 * Use logistic regression as benchmark model
 * Use sklearn pipeliens + cv + grid search with sklearn models (e.g. KNNs, RandomForest, etc.)
 * Compare all models on proper metric (your choice)

For DNN course project:
* Use sklearn pipeliens with tensorflow models (w/wo embeddings, LSTMs, RNNs, Transformers etc.)
* Compare all models on proper metric (your choice)

In [84]:
#%load_ext autoreload
#%autoreload 2
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')


In [174]:
import nltk
import sys; sys.path.append('../')

In [175]:
import string
import emoji
import re as regex
from pandas import * 

Here we will try to predict the points a wine will get based on known characteristics (i.e. features, in the ML terminology). The mine point in this stage is to establish a simple, ideally super cost effective, basline.
In the real world there is a tradeoff between complexity and perforamnce, and the DS job, among others, is to present a tradeoff tables of what performance is achivalbel at what complexity level. 

to which models with increased complexity and resource demands will be compared. Complexity should then be translated into cost. For example:
 * Compute cost 
 * Maintenance cost
 * Serving costs (i.e. is new platform needed?) 
 

#### Loading the data

In [87]:
import pandas as pd
import cufflinks as cf; cf.go_offline()
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
import seaborn as sns

In [176]:
wine_reviews = pd.read_csv("data/winemag-data-130k-v2.csv", index_col=0)
wine_reviews.shape

(129971, 13)

In [89]:
wine_reviews.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [119]:
wine_reviews.describe(include="all").transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
country,129908,43,US,54504,NaN,NaN,NaN,NaN,NaN,NaN,NaN
description,129971,119955,"Seductively tart in lemon pith, cranberry and ...",3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
designation,92506,37979,Reserve,2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN
points,129971.0,NaN,NaN,NaN,88.447138,3.03973,80.0,86.0,88.0,91.0,100.0
price,120975.0,NaN,NaN,NaN,35.363389,41.022218,4.0,17.0,25.0,42.0,3300.0
province,129908,425,California,36247,NaN,NaN,NaN,NaN,NaN,NaN,NaN
region_1,108724,1229,Napa Valley,4480,NaN,NaN,NaN,NaN,NaN,NaN,NaN
region_2,50511,17,Central Coast,11065,NaN,NaN,NaN,NaN,NaN,NaN,NaN
taster_name,103727,19,Roger Voss,25514,NaN,NaN,NaN,NaN,NaN,NaN,NaN
taster_twitter_handle,98758,15,@vossroger,25514,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Is the points normally distributed

In [91]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.hist(wine_reviews.points, edgecolor='black', bins=20, density=True)

(array([0.00305453, 0.00532426, 0.01412623, 0.02327442, 0.04985728,
        0.07332405, 0.0969447 , 0.13028291, 0.13239107, 0.09406714,
        0.11856491, 0.08739642, 0.07396265, 0.04992652, 0.02891414,
        0.01181033, 0.00402397, 0.00176193, 0.00059244, 0.00040009]),
 array([ 80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,  89.,  90.,
         91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99., 100.]),
 <BarContainer object of 20 artists>)

In [92]:
g = sns.pairplot(wine_reviews[['price','points']].sample(10000), kind="reg", diag_kind="kde")
g.fig.set_size_inches(15,15)

C:\ProgramData\Anaconda3\lib\site-packages\seaborn\axisgrid.py:1242: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).



In [93]:
from scipy.stats import shapiro 
import math
#perform Shapiro-Wilk test for normality
result = shapiro(wine_reviews.points.sample(1000))
result

ShapiroResult(statistic=0.9882988929748535, pvalue=3.716081664606463e-07)

If the p-value ≤ 0.05, then we reject the null hypothesis i.e. we assume the distribution of our variable is not normal/gaussian.
Conclusion - the points are not distributed normally

In [94]:
len(wine_reviews[wine_reviews.price.isna()])

8996

In [177]:
#replace the missing price with the midian
med = wine_reviews.price.median()
mean = wine_reviews.price.mean()
print(f'med {med} mean {mean}')


med 25.0 mean 35.363389129985535


In [178]:
#replace price null with median
price_nulls = wine_reviews.price.isna()
wine_reviews[price_nulls].price = med

#### Analize categorical columns

In [97]:
wine_reviews.nunique()

country                      43
description              119955
designation               37979
points                       21
price                       390
province                    425
region_1                   1229
region_2                     17
taster_name                  19
taster_twitter_handle        15
title                    118840
variety                     707
winery                    16757
dtype: int64

In [98]:
wine_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129971 entries, 0 to 129970
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   country                129908 non-null  object 
 1   description            129971 non-null  object 
 2   designation            92506 non-null   object 
 3   points                 129971 non-null  int64  
 4   price                  120975 non-null  float64
 5   province               129908 non-null  object 
 6   region_1               108724 non-null  object 
 7   region_2               50511 non-null   object 
 8   taster_name            103727 non-null  object 
 9   taster_twitter_handle  98758 non-null   object 
 10  title                  129971 non-null  object 
 11  variety                129970 non-null  object 
 12  winery                 129971 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 13.9+ MB


In [99]:
#country
print(wine_reviews.country.value_counts().shape)
country_count = wine_reviews.province.value_counts().sort_values(ascending=False)
country_count.head(10)

(43,)


California          36247
Washington           8639
Bordeaux             5941
Tuscany              5897
Oregon               5373
Burgundy             3980
Northern Spain       3851
Piedmont             3729
Mendoza Province     3264
Veneto               2716
Name: province, dtype: int64

In [100]:
#province
print(wine_reviews.province.value_counts().shape)
province_count = wine_reviews.province.value_counts().sort_values(ascending=False)
province_count_sum = province_count.sum()
#print(province_count.head(50))
province_count.head(50).sum()/province_count_sum

(425,)


0.9111909967053607

In [101]:
#veriety
print(wine_reviews.variety.value_counts().shape)
veriety_count = wine_reviews.variety.value_counts().sort_values(ascending=False)
veriety_count_sum = veriety_count.sum()
#print(veriety_count.head(50))
veriety_count.head(100).sum()/veriety_count_sum

(707,)


0.9471724244056321

In [102]:
print(wine_reviews.winery.value_counts().shape)
winery_count = wine_reviews.winery.value_counts().sort_values(ascending=False)
print(winery_count.head(20))
n = 1000
print(f'{n} sum: {winery_count.head(n).sum()} all sum: {winery_count.sum()} percent: {winery_count.head(n).sum()/winery_count.sum()}')
#there are many different values in the winery and small amount of reviews for each

(16757,)
Wines & Winemakers            222
Testarossa                    218
DFJ Vinhos                    215
Williams Selyem               211
Louis Latour                  199
Georges Duboeuf               196
Chateau Ste. Michelle         194
Concha y Toro                 164
Columbia Crest                159
Kendall-Jackson               130
Siduri                        126
Gary Farrell                  125
Lynmar                        118
Montes                        117
Albert Bichot                 117
Trapiche                      113
Undurraga                     113
Casa Santos Lima              113
Jean-Luc and Paul Aegerter    113
Santa Ema                     112
Name: winery, dtype: int64
1000 sum: 47584 all sum: 129971 percent: 0.3661124404674889


In [103]:
wine_reviews.columns

Index(['country', 'description', 'designation', 'points', 'price', 'province',
       'region_1', 'region_2', 'taster_name', 'taster_twitter_handle', 'title',
       'variety', 'winery'],
      dtype='object')

#### Add the Year column

In [179]:
def get_year(title):
    words = title.split(" ")
    for word in words:
        if word.isnumeric():
            num = int(word)
            if num > 1800 and num<2100:
                return num
    return 'NaN'
        
wine_reviews['year']  = wine_reviews.title.apply(get_year)
wine_reviews.sample(10)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,year
57894,France,"A solid, structured wine, showing the fine tan...",NaN,89,20.0,Bordeaux,Lussac Saint-Émilion,NaN,Roger Voss,@vossroger,Château Bel-Air 2010 Lussac Saint-Émilion,Bordeaux-style Red Blend,Château Bel-Air,2010
96611,US,Nonvintage juice at an amazing price. It's not...,NaN,83,6.0,California,California,California Other,Paul Gregutt,@paulgwine,Barefoot Cellars NV Zinfandel (California),Zinfandel,Barefoot Cellars,NaN
97198,France,"This is only Brut in name, because it is actua...",Comte de Rinken Blanc de Blancs Brut,86,14.0,Alsace,Crémant d'Alsace,NaN,Roger Voss,@vossroger,Pierre and Charles Sparr NV Comte de Rinken Bl...,Pinot Blanc,Pierre and Charles Sparr,NaN
79466,US,This Pinot is a little heavy in the mouth. It'...,Finley Vineyard,86,29.0,California,Santa Cruz Mountains,Central Coast,NaN,NaN,Cooper-Garrod 2011 Finley Vineyard Pinot Noir ...,Pinot Noir,Cooper-Garrod,2011
73427,US,Black and green olive tapenade and ripe plum c...,Mitsuko's Vineyard,86,35.0,California,Carneros,Napa-Sonoma,Virginie Boone,@vboone,Clos Pegase 2010 Mitsuko's Vineyard Merlot (Ca...,Merlot,Clos Pegase,2010
44914,Chile,"Chunky, meaty, saucy plum and berry aromas are...",Special Reserve,85,16.0,Colchagua Valley,NaN,NaN,Michael Schachner,@wineschach,Siegel 2013 Special Reserve Cabernet Sauvignon...,Cabernet Sauvignon,Siegel,2013
61265,US,A delicious Pinot Noir. Set it beside more exp...,NaN,87,14.0,California,Central Coast,Central Coast,NaN,NaN,Cupcake 2008 Pinot Noir (Central Coast),Pinot Noir,Cupcake,2008
64277,France,"A rich, sweet wine, showing the great improvem...",NaN,91,NaN,Bordeaux,Pomerol,NaN,Roger Voss,@vossroger,Château Petit-Village 2006 Pomerol,Bordeaux-style Red Blend,Château Petit-Village,2006
122667,France,"French actor Gérard Depardieu, who has a viney...",Taille Princesse de Gérard Depardieu Rosé Brut,89,NaN,Loire Valley,Saumur,NaN,Roger Voss,@vossroger,Bouvet-Ladubay 2008 Taille Princesse de Gérard...,Sparkling Blend,Bouvet-Ladubay,2008
63500,Italy,"A simple but well made Dolcetto, with aromas o...",Vigna Ciabot Russ,85,20.0,Piedmont,Dolcetto d'Alba,NaN,Kerin O’Keefe,@kerinokeefe,Mario Gagliasso 2011 Vigna Ciabot Russ (Dolce...,Dolcetto,Mario Gagliasso,2011


In [180]:
g = sns.pairplot(wine_reviews[['price','points', 'year']].sample(10000), kind="reg", diag_kind="kde")
g.fig.set_size_inches(15,15)

In [181]:
wine_reviews.year[wine_reviews.year.isna()] = 1800
wine_reviews.sample(5)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,year
81266,US,"Shows a tightly wound structure, with big, har...",GOTT 7,94,50.0,California,Napa Valley,Napa,NaN,NaN,Joel Gott 2007 GOTT 7 Cabernet Sauvignon (Napa...,Cabernet Sauvignon,Joel Gott,2007
4422,US,"Very dark in color, this robust wine has entic...",NaN,89,28.0,California,Sierra Foothills,Sierra Foothills,Jim Gordon,@gordone_cellars,Le Casque 2014 Petite Sirah (Sierra Foothills),Petite Sirah,Le Casque,2014
120435,US,"Quite light in the glass, this 50-50 blend fro...",Aloysius,93,39.0,California,Sta. Rita Hills,Central Coast,Matt Kettmann,@mattkettmann,Jamie Slone Wines 2014 Aloysius Chardonnay (St...,Chardonnay,Jamie Slone Wines,2014
2536,US,"Dried blueberry fruit, asphalt, black plums an...",NaN,89,29.0,California,Santa Barbara County,Central Coast,Matt Kettmann,@mattkettmann,Jaffurs 2013 Syrah (Santa Barbara County),Syrah,Jaffurs,2013
108827,Austria,There are just a few hints of pepper on this c...,NaN,87,15.0,Burgenland,NaN,NaN,Anne Krebiehl MW,@AnneInVino,Meinklang 2015 Grüner Veltliner (Burgenland),Grüner Veltliner,Meinklang,2015


In [182]:
wine_reviews = wine_reviews.dropna(subset={'country', 'variety'})
wine_reviews.drop(['taster_twitter_handle','title','designation','region_2','region_1'],axis=1, inplace=True)
wine_reviews.drop(['winery'], axis = 1, inplace=True)

In [183]:
wine_reviews.drop_duplicates(inplace=True)

In [184]:
wine_reviews.to_csv("data/wine_reviews.csv")

#### Remove categorical columns with too much categoris

In [108]:
reviews = wine_reviews

province_to_keep = province_count.head(50).index.array
reviews.province = reviews.province.apply(lambda x: x if x in province_to_keep else 'unknown')

veriety_to_keep = veriety_count.head(100).index.array
reviews.variety = reviews.variety.apply(lambda x: x if x in veriety_to_keep else 'unknown')

print(reviews.province.value_counts().sort_values(ascending=False).head(10))
print(reviews.variety.value_counts().sort_values(ascending=False).head(10))

California          36247
unknown             11537
Washington           8639
Bordeaux             5941
Tuscany              5897
Oregon               5373
Burgundy             3980
Northern Spain       3851
Piedmont             3729
Mendoza Province     3264
Name: province, dtype: int64
Pinot Noir                  13269
Chardonnay                  11750
Cabernet Sauvignon           9470
Red Blend                    8935
Bordeaux-style Red Blend     6915
unknown                      6845
Riesling                     5188
Sauvignon Blanc              4963
Syrah                        4142
Rosé                         3563
Name: variety, dtype: int64
